In [1]:
import pandas as pd

movies = pd.read_csv('ml-latest-small/movies.csv')
ratings = pd.read_csv('ml-latest-small/ratings.csv')
tags = pd.read_csv('ml-latest-small/tags.csv', encoding='latin1')  # если есть проблемы с кодировкой

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Преобразуем жанры в строки
movies['genres'] = movies['genres'].apply(lambda x: x.replace('|', ' '))

# Создаем TF-IDF матрицу для жанров
tfidf_genres = TfidfVectorizer()
tfidf_genres_matrix = tfidf_genres.fit_transform(movies['genres'])

In [3]:
# Агрегируем теги по movieId
movie_tags = tags.groupby('movieId')['tag'].apply(lambda x: ' '.join(x)).reset_index()

# Объединяем с movies
movies = movies.merge(movie_tags, on='movieId', how='left')
movies['tag'] = movies['tag'].fillna('')  # заменяем NaN на пустые строки

# TF-IDF для тегов
tfidf_tags = TfidfVectorizer()
tfidf_tags_matrix = tfidf_tags.fit_transform(movies['tag'])

In [4]:
# Статистики по пользователям
user_stats = ratings.groupby('userId')['rating'].agg(['mean', 'median', 'var']).reset_index()
user_stats.columns = ['userId', 'user_mean_rating', 'user_median_rating', 'user_var_rating']

# Статистики по фильмам
movie_stats = ratings.groupby('movieId')['rating'].agg(['mean', 'median', 'var']).reset_index()
movie_stats.columns = ['movieId', 'movie_mean_rating', 'movie_median_rating', 'movie_var_rating']

# Объединяем с основной таблицей ratings
data = ratings.merge(user_stats, on='userId', how='left')
data = data.merge(movie_stats, on='movieId', how='left')

In [5]:
from scipy.sparse import hstack

# Получаем TF-IDF фичи для всех movieId в data
movie_indices = data['movieId'].apply(lambda x: movies.index[movies['movieId'] == x][0])
tfidf_genres_features = tfidf_genres_matrix[movie_indices]
tfidf_tags_features = tfidf_tags_matrix[movie_indices]

# Объединяем все фичи в одну матрицу
X = hstack([
    tfidf_genres_features,
    tfidf_tags_features,
    data[['user_mean_rating', 'user_median_rating', 'user_var_rating',
          'movie_mean_rating', 'movie_median_rating', 'movie_var_rating']].values
])
y = data['rating'].values

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import HistGradientBoostingRegressor

from sklearn.metrics import root_mean_squared_error
from scipy.sparse import issparse
import numpy as np

# Проверяем и преобразуем X в плотную матрицу, если нужно
if issparse(X):
    X = X.toarray()

# Заменяем оставшиеся NaN (если есть)
X = np.nan_to_num(X)

# Разделяем данные
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Обучаем модель
model = HistGradientBoostingRegressor(random_state=42)
model.fit(X_train, y_train)

# Предсказываем и оцениваем
y_pred = model.predict(X_test)
rmse = root_mean_squared_error(y_test, y_pred)
print(f'RMSE: {rmse}')

c:\anaconda\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] Не удается найти указанный файл
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\anaconda\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "c:\anaconda\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\anaconda\Lib\subprocess.py", line 1026, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "c:\anaconda\Lib\subprocess.py", line 1538, in _execute_child
    hp, ht, pid, tid = _winapi.CreateProcess(executable, args,
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

RMSE: 0.7925383109914156
